In [1]:
from jwp.jwcsv import *
from jwp.jwcache import *
from jwp.jwsoup import *
import billboard


In [75]:
""""Billboard utils - done and moved into own file"
links_dont_count = False
artist_handles = {}
def format_entry_as_tweet(source, chart, entry):
    """Returns an entry formatted as a tweet <140ch
    Args:
        string source: name of the source (eg billboard)
        string chart: name of chart (eg Hot-100)"""
    if chart == 'billboard-200':
        twstr = 'US Top Albums: '
    else:
        twstr = source + ' ' + chart.title() + ': '
    # add rank and change
    twstr = twstr + '#' + str(entry.rank) + format_change(entry.change)
    # add title
    twstr = twstr + ' ' + entry.title + ','
    # add artist
    twstr = twstr + ' ' + replace_artist_handle(entry.artist)
    # add weeks
    twstr = twstr + ' ' + format_weeks(entry.weeks)
    twstr = twstr.strip()
    # special stuff
    twstr = twstr + ' ' + hotshot_suffix(entry.change)
    twstr = twstr.strip()
    twstr = twstr + ' ' + peak_suffix(entry)
    twstr = twstr.strip()
    if len(twstr) > 140:
        twstr = compensate(twstr, entry.artist)
    if links_dont_count:
        twstr = twstr + ' ' + link(chart, entry.spotifyID)
    return twstr.strip()

def format_change(change):
    if change == '0':
        return '(=)'
    elif change == 'New':
        return '(new)'
    elif change == 'Re-Entry':
        return '(re-entry)'
    elif any(change.startswith(x) for x in ['+', '-']):
        return '(' + change + ')'
    return '(new)'

def format_weeks(weeks):
    if weeks > 1:
        return '[{0} weeks]'.format(weeks)
    return ''

def format_artist(artist):
    """Returns a string of artists/featured artists, replacing with handle
    where necessary.
    Args:
        string artist: artist property of entry object"""
    artist = format_feat(artist)
    # split on featured
    featured_artists = artist.split(' feat. ')
    # if contains a featured artist, figure that out
    if len(featured_artists) > 1:
        # create a featured list 
        featured_list = []
        artist = featured_artists[0]
        featured = format_ampersand(featured_artists[1]).split(',')
        for a in featured_list:
            featured_list.append(artist_handles.get(artist, artist))
        main_handle = artist_handles.get(artist, artist)
        return main_handle + ' feat. ' + ', '.join(featured_list)
    return artist_handles.get(artist, artist)

def hotshot_suffix(change):
    if change == 'Hot Shot Debut':
        return '*Hot Shot Debut*'
    return ''

def peak_suffix(entry):
    "Returns *new peak* if ostensibly the new peak"
    "TODO: create a dict of name artist: peak and check that way?"
    peak = entry.peakPos
    change = entry.change
    rank = entry.rank
    if change.startswith('+'):
        if rank == peak:
            return '*new peak*'     
    return ''
    
def format_feat(artist):
    "Formats featuring, Featuring, feat., feat, etc to a uniform string"
    artist.replace('Featuring', 'feat.')
    artist.replace('featuring', 'feat.')
    artist.replace(' feat ', ' feat. ')
    return artist

def format_ampersand(artist_split):
    "Replace ampersand with a comma for featured artists for ease of splitting"
    if '&' in artist_split and ',' in artist_split:
        return artist_split.replace('&', ',')
    return artist_split

def compensate(twstr, artist):
    "Compensates for long artist strings"
    artist_format = format_artist(artist)
    over = len(twstr) - 140
    replace_str = artist_format[:-over-2] + '...'
    return twstr.replace(artist_format, replace_str)

def link(chart, spid):
    if not spid:
        return ''
    open_link = 'https://open.spotify.com/'
    if chart == 'billboard-200':
        open_link = open_link + 'album/' + spid
        return open_link
    open_link = open_link + 'track/' + spid
    return open_link"""

Billboard Hot-100: #1(=) One Dance, Drake feat. WizKid & Kyla [13 weeks]
Billboard Hot-100: #2(=) Can't Stop The Feeling!, Justin Timberlake [8 weeks]
Billboard Hot-100: #3(+1) Don't Let Me Down, The Chainsmokers feat. Daya [20 weeks] *new peak*
Billboard Hot-100: #4(-1) Panda, Desiigner [19 weeks]
Billboard Hot-100: #5(=) This Is What You Came For, Calvin Harris feat. Rihanna [9 weeks]
Billboard Hot-100: #6(=) Cheap Thrills, Sia feat. Sean Paul [20 weeks]
Billboard Hot-100: #7(+1) Needed Me, Rihanna [22 weeks] *new peak*
Billboard Hot-100: #8(+1) Don't Mind, Kent Jones [9 weeks] *new peak*
Billboard Hot-100: #9(+1) Ride, twenty one pilots [16 weeks] *new peak*
Billboard Hot-100: #10(-3) Work From Home, Fifth Harmony feat. Ty Dolla $ign [18 weeks]
Billboard Hot-100: #11(=) I Took A Pill In Ibiza, Mike Posner [24 weeks]
Billboard Hot-100: #12(=) Just Like Fire, P!nk [11 weeks]
Billboard Hot-100: #13(+6) Send My Love (To Your New Lover), Adele [7 weeks] *new peak*
Billboard Hot-100: #14(

Billboard Hot-100: #1(=) One Dance, Drake feat. WizKid & Kyla [13 weeks]
Billboard Hot-100: #2(=) Can't Stop The Feeling!, Justin Timberlake [8 weeks]
Billboard Hot-100: #3(+1) Don't Let Me Down, The Chainsmokers feat. Daya [20 weeks] *new peak*
Billboard Hot-100: #4(-1) Panda, Desiigner [19 weeks]
Billboard Hot-100: #5(=) This Is What You Came For, Calvin Harris feat. Rihanna [9 weeks]
Billboard Hot-100: #6(=) Cheap Thrills, Sia feat. Sean Paul [20 weeks]
Billboard Hot-100: #7(+1) Needed Me, Rihanna [22 weeks] *new peak*
Billboard Hot-100: #8(+1) Don't Mind, Kent Jones [9 weeks] *new peak*
Billboard Hot-100: #9(+1) Ride, twenty one pilots [16 weeks] *new peak*
Billboard Hot-100: #10(-3) Work From Home, Fifth Harmony feat. Ty Dolla $ign [18 weeks]
Billboard Hot-100: #11(=) I Took A Pill In Ibiza, Mike Posner [24 weeks]
Billboard Hot-100: #12(=) Just Like Fire, P!nk [11 weeks]
Billboard Hot-100: #13(+6) Send My Love (To Your New Lover), Adele [7 weeks] *new peak*
Billboard Hot-100: #14(